**Avatars IDs**

- Mens US ASTM Sz M/40, MALE ID: "b4ec391f-4a63-41ad-8efd-f3e45c75bb28"

- Womens US ASTM Sz M/6, FEMALE ID: "ad80d4fd-cf03-4337-b42a-8df04ea07bcc"

# *Compute new positions of the garment*

In [385]:
import json
#IMPORT SCENE
rute = 'store_state_transformed.json'

with open(rute) as archivo:
    data = json.load(archivo)

# Avatar & Pieces geometry data
avatar = data["avatar"]
pieces = data["pieces"]
#print(pieces)
#print("Avatar", avatar)
all_pieces_indices = []
all_pieces_positon3d = []
for piece in data['pieces']:
    print(piece)
    #if(piece['_id']=='66016a04a0216e4eb0a641b3'):
    indices = piece['components']['garmentPiece']['geometry']['indices']
    posiciones = piece['components']['garmentPiece']['geometry']['position3D']
    all_pieces_indices.extend(indices)
    all_pieces_positon3d.extend(posiciones)



#print(len(all_pieces_positon3d))
print("Pieces", len(all_pieces_positon3d))

IndentationError: unexpected indent (247075532.py, line 18)

In [ ]:
import numpy as np

#Convert to numpy arrays & matrix
avatar_indices = np.array(avatar["indices"])
avatar_positions = np.array(avatar["positions"]).reshape(-1, 3)  # Convertir a una matriz de 3 columnas

pieces_indices = np.array(all_pieces_indices)
pieces_positions = np.array(all_pieces_positon3d).reshape(-1, 3)  # Convertir a una matriz de 3 columnas

#print(len(pieces_positions))

In [ ]:
# Compute distancias minimas prenda-avatar
distancias_minimas = np.zeros(len(pieces_positions))
ac = np.zeros(len(pieces_positions), dtype=int) # AvatarCorrespondeces

# For all vertex in the garment
for i, pos_prenda in enumerate(pieces_positions):
    distancias_vertice = np.linalg.norm(pos_prenda - avatar_positions, axis=1)
    # Find nearest avatar index and compute min distance
    indice_vertice_cercano = np.argmin(distancias_vertice)
    distancia_minima = distancias_vertice[indice_vertice_cercano]
    print(distancias_vertice[indice_vertice_cercano])

    # Save min distances and avatar index correspondences
    distancias_minimas[i] = distancia_minima
    ac[i] = indice_vertice_cercano

#print("La distancia mínimas entre el avatar y la geometría de las piezas de ropa", distancias_minimas,ac)

0.007093759407071145
0.007869449565303442
0.008559315718357503
0.00854430863620998
0.010069760630833922
0.012541059434487946
0.011706392577060966
0.011764790067127832
0.010616449441917987
0.007666861424541548
0.007941284063258777
0.006269595622286067
0.003560636529935408
0.00798964687184997
0.0029421984058888894
0.009295229041720119
0.005563796028563971
0.010766723338719284
0.015890430981775238
0.024042303551749328
0.025859115162914007
0.02665342321486511
0.02884141738612792
0.02922461771157768
0.027190915762883714
0.02519619574765959
0.025185393956022598
0.024052644440744522
0.021407073115862305
0.022371455816212717
0.021058666353791188
0.019195406642210856
0.02124270046546261
0.024631443960189247
0.02669734927457329
0.025201449466520576
0.02016382869224082
0.018032143101819012
0.01725115789951548
0.014989177246548078
0.014475660098236982
0.012595689442385535
0.011599227601573595
0.013342097282635244
0.014270153360686811
0.010524034894035693
0.010954997273549162
0.007288515711370829
0

In [ ]:
# Offset between garment vertex and its nearest avatar vertex
offsets = np.zeros_like(pieces_positions)

for i, indice_avatar in enumerate(ac):
    offset = np.array(pieces_positions[i])- np.array(avatar_positions[indice_avatar])  
    offsets[i]=offset

print(offsets)



[[ 5.43050468e-04  3.49557400e-03  6.14877883e-03]
 [ 2.87202001e-03  4.02402878e-03  6.12265710e-03]
 [ 4.72535938e-03  3.14188004e-03  6.40792120e-03]
 ...
 [ 1.20823234e-02 -1.82569027e-04  1.67916715e-02]
 [-1.12503543e-02 -4.50015068e-05  1.70839950e-02]
 [ 8.12505186e-03  1.21700764e-02  1.56131089e-02]]


New Avatar

In [ ]:
#IMPORT NEW AVATAR
avatars = 'avatars.json'

with open(avatars) as file:
    avatars_data = json.load(file)
new_avatar = avatars_data["male"]

new_avatar_positions= np.array(new_avatar["positions"]).reshape(-1, 3)  # Convertir a una matriz de 3 columnas


In [ ]:
#Compute new pieces positions with the offset computed and its nearest avatar index from the new avatar (Avatars in seddi have the same mesh)
new_pieces_positions = np.zeros_like(pieces_positions)

for i, indice_avatar in enumerate(ac):
    new_pieces_positions[i] = np.array(new_avatar_positions[indice_avatar]) + np.array(offsets[i])


#print(len(new_pieces_positions))


In [ ]:
#Export SCENE TO JSON for AUTHOR

import json
#IMPORT SCENE
rute = 'store_state_transformed.json'

with open(rute) as archivo:
    data = json.load(archivo)


pieces = data["pieces"]
new_list_posiciones = new_pieces_positions.reshape(-1).tolist()

for piece in data['pieces']:
    #if(piece['_id']=='66016a04a0216e4eb0a641b3'):
    indices = piece['components']['garmentPiece']['geometry']['indices']
    posiciones = piece['components']['garmentPiece']['geometry']['position3D']

    # Determinar la longitud necesaria para la pieza actual
    longitud_pieza = len(posiciones)
    # Obtener las nuevas posiciones para la pieza actual
    nuevas_posiciones = new_list_posiciones[:longitud_pieza]

    # Reemplazar las posiciones originales por las nuevas posiciones
    piece['components']['garmentPiece']['geometry']['position3D'] = nuevas_posiciones
    
    # Actualizar new_pieces_positions para la próxima pieza
    new_list_posiciones = new_list_posiciones[longitud_pieza:]

with open("output.json", "w") as outfile: 
      json.dump(data, outfile)
print("Datos exportados a 'output.json'")

Datos exportados a 'output.json'
